In [33]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array
from time import time

In [34]:
# load data

df=pd.read_csv('Annthyroid_02_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.026,0.139,0.090,0.153,0


In [35]:
df.shape

(6802, 22)

In [36]:
# df to values
df = df.values

In [37]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Annthyroid.json")  
gc = GCForest(config)

In [38]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [39]:
# count class
len(np.unique(y_train))

2

In [40]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-29 17:53:04,668][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 21)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 21)],y_test.shape=(2041,)
[ 2018-04-29 17:53:04,670][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-29 17:53:04,672][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 17:53:04,674][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-29 17:53:04,675][cascade_classifier.fit_transform] X_train.shape=(4761, 21),X_test.shape=(2041, 21)
[ 2018-04-29 17:53:04,677][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 21), X_cur_test.shape=(2041, 21)
[ 2018-04-29 17:53:05,500][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.74%
[ 2018-04-29 17:53:06,590][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.79%
[ 2018-04-29 17:53:07,727][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [41]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-29 17:53:15,726][cascade_classifier.transform] X_groups_test.shape=[(2041, 21)]
[ 2018-04-29 17:53:15,728][cascade_classifier.transform] group_dims=[21]
[ 2018-04-29 17:53:15,730][cascade_classifier.transform] X_test.shape=(2041, 21)
[ 2018-04-29 17:53:15,732][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2041, 21)


Test Accuracy of ExtratTrees = 98.971093 %


In [42]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-29 17:53:18,175][cascade_classifier.transform] X_groups_test.shape=[(2041, 21)]
[ 2018-04-29 17:53:18,176][cascade_classifier.transform] group_dims=[21]
[ 2018-04-29 17:53:18,179][cascade_classifier.transform] X_test.shape=(2041, 21)
[ 2018-04-29 17:53:18,181][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2041, 21)


In [43]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [44]:
X_test_new.shape

(2041, 23)

In [45]:
X_train_new.shape

(4761, 23)

In [46]:
# Cascade ExtraTrees
config = load_json("./examples/Annthyroidgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-29 17:53:20,320][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 23)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 23)],y_test.shape=(2041,)
[ 2018-04-29 17:53:20,322][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-29 17:53:20,323][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 17:53:20,325][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-29 17:53:20,326][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-29 17:53:20,328][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)
[ 2018-04-29 17:53:21,173][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.37%
[ 2018-04-29 17:53:22,108][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-29 17:53:23,218][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

[ 2018-04-29 17:54:06,749][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.37%
[ 2018-04-29 17:54:07,832][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.53%
[ 2018-04-29 17:54:09,049][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=98.32%
[ 2018-04-29 17:54:10,110][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.37%
[ 2018-04-29 17:54:11,178][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=98.74%
[ 2018-04-29 17:54:12,305][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.58%
[ 2018-04-29 17:54:13,583][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.79%
[ 2018-04-29 17:54:14,720][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.16%


In [47]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [48]:
for i in range (20):
    t0=time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/Annthyroidgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-29 17:54:37,045][cascade_classifier.fit_transform] X_groups_train.shape=[(4761, 23)],y_train.shape=(4761,),X_groups_test.shape=[(2041, 23)],y_test.shape=(2041,)
[ 2018-04-29 17:54:37,047][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-29 17:54:37,048][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 17:54:37,049][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-29 17:54:37,051][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-29 17:54:37,052][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)
[ 2018-04-29 17:54:37,939][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.16%
[ 2018-04-29 17:54:39,034][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-29 17:54:40,134][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

[ 2018-04-29 17:55:22,488][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.16%
[ 2018-04-29 17:55:23,589][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.11%
[ 2018-04-29 17:55:24,709][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=98.74%
[ 2018-04-29 17:55:25,809][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=98.95%
[ 2018-04-29 17:55:26,920][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=98.53%
[ 2018-04-29 17:55:27,938][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.16%
[ 2018-04-29 17:55:29,027][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.58%
[ 2018-04-29 17:55:30,143][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.16%


('Test Accuracy of Extratrees = 99.461049 %', 0, 'fois')
(' Time ', '54.543', ' seconds')


[ 2018-04-29 17:55:37,632][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.16%
[ 2018-04-29 17:55:38,727][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.16%
[ 2018-04-29 17:55:39,882][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.37%
[ 2018-04-29 17:55:40,964][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.74%
[ 2018-04-29 17:55:42,083][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.95%
[ 2018-04-29 17:55:43,312][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.53%
[ 2018-04-29 17:55:44,297][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.74%
[ 2018-04-29 17:55:45,434][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.74%


[ 2018-04-29 17:56:27,378][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.95%
[ 2018-04-29 17:56:28,443][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.95%
[ 2018-04-29 17:56:29,604][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.95%
[ 2018-04-29 17:56:30,718][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.16%
[ 2018-04-29 17:56:30,980][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.91%
[ 2018-04-29 17:56:30,982][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=99.36%
[ 2018-04-29 17:56:30,983][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.91%
[ 2018-04-29 17:56:30,985][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=99.36%
[ 2018-04-29 17:56:30,986][

...

[ 2018-04-29 18:15:32,237][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.16%
[ 2018-04-29 18:15:33,319][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.58%
[ 2018-04-29 18:15:34,275][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.16%
[ 2018-04-29 18:15:35,320][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.32%
[ 2018-04-29 18:15:36,472][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.74%
[ 2018-04-29 18:15:37,592][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.95%
[ 2018-04-29 18:15:38,701][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.37%
[ 2018-04-29 18:15:39,780][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.95%


[ 2018-04-29 18:16:16,377][cascade_classifier.fit_transform] X_train.shape=(4761, 23),X_test.shape=(2041, 23)
[ 2018-04-29 18:16:16,379][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4761, 23), X_cur_test.shape=(2041, 23)


('Test Accuracy of Extratrees = 98.824106 %', 18, 'fois')
(' Time ', '42.502', ' seconds')


[ 2018-04-29 18:16:17,226][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.74%
[ 2018-04-29 18:16:18,354][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.37%
[ 2018-04-29 18:16:19,523][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.16%
[ 2018-04-29 18:16:20,605][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.32%
[ 2018-04-29 18:16:21,627][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.53%
[ 2018-04-29 18:16:22,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.16%
[ 2018-04-29 18:16:23,808][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.95%
[ 2018-04-29 18:16:24,935][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.95%


('Test Accuracy of Extratrees = 98.873101 %', 19, 'fois')
(' Time ', '43.629', ' seconds')


# ('Test Accuracy of Extratrees = 99.461049 %', 0, 'fois')
(' Time ', '54.543', ' seconds')